In [ ]:
####### NEW VERSION

        # Define the cost function
        ocp.cost.cost_type = 'LINEAR_LS'

        # Define the Q and R matrices separately for clarity and reuse
        max_x_err = 0.01
        max_y_err = 0.01
        max_z_err = 0.01
        max_vx_err = 0.1
        max_vy_err = 0.1
        max_vz_err = 0.1
        max_roll_err = 0.1
        max_pitch_err = 0.01
        max_yaw_err = 0.1
        max_thrust = 27.0
        max_rate = 0.8


        # Q = 50* np.diag([20., 20., 20., 2., 2., 2., 1., 1., 20.])  # weights for states
        Q = 150 * np.diag([1/(max_x_err**2), 1/(max_y_err**2), 1/(max_z_err**2), 1/(max_vx_err**2), 1/(max_vy_err**2), 1/(max_vz_err**2), 1/(max_roll_err**2), 1/(max_pitch_err**2), 10/(max_yaw_err**2)])  # weights for states
        # R = np.diag([1., 1., 1., 1.])  # weights for controls
        R = np.diag([1/(max_thrust**2), 1/(max_rate**2), 1/(max_rate**2), 1/(max_rate**2)])  # weights for controls

        # Construct the Vx and Vu matrices with appropriate dimensions
        ocp.cost.Vx = np.vstack([np.identity(9), np.zeros((4, 9))])  # 9x9 identity for states, padded for controls
        ocp.cost.Vu = np.vstack([np.zeros((9, 4)), np.identity(4)])  # zero for states, 4x4 identity for controls

        # Combine Q and R into a block diagonal matrix W for state and control errors
        ocp.cost.W = np.block([[Q, np.zeros((9, 4))], [np.zeros((4, 9)), R]])

        # Reference vector for states and controls, sized according to W
        ocp.cost.yref = np.zeros(13)  # 9 states + 4 controls

        # Define the terminal cost using the same structure
        ocp.cost.cost_type_e = 'LINEAR_LS'
        ocp.cost.W_e = Q  # Terminal cost weight for states
        ocp.cost.Vx_e = np.identity(9)  # 9x9 identity for states
        ocp.cost.yref_e = np.zeros(9)  # Terminal reference for states

In [ ]:
    def solve_mpc_control_other(self, x0, yref):
        N = self.num_steps
        nx = len(x0)
        nu = 4
        
        if yref.shape[1] != self.num_steps:
            raise Exception('incorrect size of yref')
    
        for i in range(N):
            self.ocp_solver.set(i, 'yref', np.array([*yref[:,i], *self.hover_control]))
            # self.ocp_solver.set(i, 'x', yref[:,i])
            # self.ocp_solver.set(i, 'u', self.hover_control)
        
        # self.ocp_solver.set(N, 'yref', yref_e)

        x_mpc = np.zeros((N+1, nx))
        u_mpc = np.zeros((N, nu))
        self.ocp_solver.set(0, 'lbx', x0)
        self.ocp_solver.set(0, 'ubx', x0)
        
        status = self.ocp_solver.solve() # solve the optimization problem
        x_mpc = self.ocp_solver.get(0, 'x') # get the optimal state trajectory
        u_mpc = self.ocp_solver.get(0, 'u') # get the optimal control trajectory



        return status, x_mpc, u_mpc